In [1]:
# Install OR-Tools if not installed
# %pip install ortools

from ortools.sat.python import cp_model

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 20.2 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 135 kB 28.9 MB/s eta 0:00:01
     |████████████████████████████████| 425 kB 33.3 MB/s eta 0:00:01
     |████████████████████████████████| 10.8 MB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 347 kB 20.8 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


ImportError: dlopen(/Users/edwardbrady/Library/Python/3.9/lib/python/site-packages/ortools/sat/python/cp_model_helper.so, 0x0002): Library not loaded: /Users/corentinl/work/stable/temp_python3.9/lib/libscip.9.2.dylib
  Referenced from: <07FF885B-3BED-30C4-9BE0-B2A889DC2E5D> /Users/edwardbrady/Library/Python/3.9/lib/python/site-packages/ortools/sat/python/cp_model_helper.so
  Reason: tried: '/Users/corentinl/work/stable/temp_python3.9/lib/libscip.9.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/corentinl/work/stable/temp_python3.9/lib/libscip.9.2.dylib' (no such file), '/Users/corentinl/work/stable/temp_python3.9/lib/libscip.9.2.dylib' (no such file)

In [1]:
# Sample data

exams = ['Math', 'Physics', 'Chemistry', 'History']
students = {
    'Alice': ['Math', 'Physics'],
    'Bob': ['Physics', 'Chemistry'],
    'Charlie': ['Math', 'History'],
}
days = ['Mon', 'Tue', 'Wed']

# Fixed exam slots (exam: day index)
fixed_slots = {
    'History': 2  # History exam fixed on Wed
}

In [3]:
for student in students:
    print(student)

Alice
Bob
Charlie


In [ ]:

model = cp_model.CpModel()

In [ ]:
# Create variables:
# exam_schedule[exam] = day assigned (0..len(days)-1)
exam_schedule = {}
for exam in exams:
    if exam in fixed_slots:
        # Fixed day
        exam_schedule[exam] = model.NewConstant(fixed_slots[exam])
    else:
        exam_schedule[exam] = model.NewIntVar(0, len(days)-1, exam)


In [ ]:

# Constraint 1: No student has more than 1 exam on the same day
for student, stu_exams in students.items():
    for i in range(len(stu_exams)):
        for j in range(i+1, len(stu_exams)):
            # exams for this student must be on different days
            model.Add(exam_schedule[stu_exams[i]] != exam_schedule[stu_exams[j]])

In [ ]:

# Constraint 2: No more than 2 exams per day (optional example)
for d in range(len(days)):
    exams_on_day = []
    for exam in exams:
        is_on_day = model.NewBoolVar(f'{exam}_on_day_{d}')
        model.Add(exam_schedule[exam] == d).OnlyEnforceIf(is_on_day)
        model.Add(exam_schedule[exam] != d).OnlyEnforceIf(is_on_day.Not())
        exams_on_day.append(is_on_day)
    model.Add(sum(exams_on_day) <= 2)


In [ ]:
# Solve model
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
    print("Exam schedule:")
    for exam in exams:
        assigned_day = solver.Value(exam_schedule[exam])
        print(f" - {exam}: {days[assigned_day]}")
else:
    print("No solution found.")
